In [ ]:
import os
from tqdm.auto import tqdm

import datasets
from datasets import Dataset, DatasetDict, load_dataset

from omegaconf import OmegaConf
from omegaconf.dictconfig import DictConfig

from accelerate import Accelerator

from datasets import load_dataset

import transformers
from transformers import (
    AutoTokenizer,
    LlamaTokenizer
)

# import bittensor

In [7]:
# base_url = "https://the-eye.eu/public/AI/pile/"
base_url = '/mnt/share/the_pile/'
data_files = {
    # "train": [base_url + "train/" + f"{idx:02d}.jsonl.zst" for idx in range(1)],
    "validation": base_url + "val.jsonl.zst",
    # "test": base_url + "test.jsonl.zst",
}

data_files

{'train': '/mnt/share/the_pile/val.jsonl.zst'}

In [11]:
from datasets import load_dataset

pile_dataset = load_dataset("json", data_files=data_files)
# pile_dataset_streamed = load_dataset("json", data_files=data_files, streaming=True)
# next(iter(pile_dataset["train"]))

Extracting data files: 100%|██████████| 1/1 [00:06<00:00,  6.05s/it]


Dataset json downloaded and prepared to /home/paperspace/.cache/huggingface/datasets/json/default-9a2dd9e55d64cca5/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 195.64it/s]


In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf", config="/mnt/share/llama-7b-hf/tokenizer_config.json", cache_dir='/mnt/share/.cache/huggingface/hub')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [17]:
cfg = OmegaConf.load('conf/config.yaml')

column_names = pile_dataset.column_names
text_column_name = "text" if "text" in column_names else column_names["train"][0]
if cfg.dataset.concatenate_raw is True:
    pad = False
else:
    pad = "max_length"

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= cfg.dataset.block_size:
        total_length = (
            total_length // cfg.dataset.block_size
        ) * cfg.dataset.block_size
    # Split by chunks of max_len.
    result = {
        k: [
            t[i : i + cfg.dataset.block_size]
            for i in range(0, total_length, cfg.dataset.block_size)
        ]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

def tokenize_fn(examples):
    result = tokenizer(
        examples[text_column_name],
        padding=pad,
        truncation=True,
        max_length=cfg.dataset.block_size,
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_datasets = pile_dataset.map(
        tokenize_fn,
        batched=True,
        num_proc=cfg.tokenizer.preprocessing_num_workers,
        load_from_cache_file=not cfg.dataset.overwrite_cache,
        desc="Running tokenizer on dataset",
        )

In [19]:
cols_to_remove = list((tokenized_datasets['train'].column_names))
cols_to_remove.remove('input_ids')
cols_to_remove.remove('attention_mask')
cols_to_remove.remove('labels')
tokenized_datasets = tokenized_datasets.remove_columns(cols_to_remove)

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 214670
    })
})

In [21]:
tokenized_datasets.save_to_disk('/mnt/share/the_pile/tokenized_val')

In [26]:
from datasets import load_from_disk

loaded_dataset = load_from_disk("/mnt/share/the_pile/tokenized_val")['train'].with_format(None).to_iterable_dataset()

In [27]:
next(iter(loaded_dataset)).keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])